# HAMILTONIANS

In [ ]:
# default_exp Hamiltonians

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export

import numpy as np
from mpmath import *

# 1. LONG RANGE AND QUASI-PERIODIC KITAEV CHAIN - HAMILTONIANS

In order to obtain the Hamiltonian in the momentum space, we start from the follwing Hamiltonian:

\begin{equation}\label{eq: hamiltonian}
    H = \sum_{n=0}^{N-1} \left [t \left (c^\dagger_{n+1}c_{n} + c^\dagger_{n}c_{n+1}\right) - \mu f(n)\left ( 2c^\dagger_{n}c_{n}-1\right) + \sum_{l=1}^{N-1}\frac{\Delta}{d^\alpha_l}\left (c^\dagger_{n+l}c^\dagger_{n} + c_{n}c_{n+l}\right)\right],
\end{equation}

where $t$ is the hopping amplitude, $\mu$ is the chemical potential, $\Delta$ is the superconducting pairing amplitude and $c_{n}(c^\dagger_{n})$ are the annihilation (creation) operators at the $n$th site of the chain. Consider we have a $N$ sites in the chain and the function $f(n) = 1-\cos\left (2 \pi n \frac{F_{n-1}}{F_{n}} + \phi \right )$, where $F_{n-1},F_n$ are integer from the Fibonacci sequence and $\phi$ is the phase. Therefore, our Hamiltonian has a periodicity of $F_n = q$ sites. 

## CALCULATIONS

### REAL SPACE HAMILTONIAN WITH APBC

Let's consider the following basis $\chi = \left(c_{0}, c^\dagger_{0}, c_{1}, c^\dagger_{1}, ..., c_{N-1}, c^\dagger_{N-1}\right)^T$. The Hamiltonian can be expressed as follows:

\begin{equation}
    H = \chi^\dagger H_N \chi
\end{equation}
where:
\begin{equation}
H_N = 
\begin{pmatrix}
A_0 & B & C_2 & C_3 & \cdots & C_{N-2} & -B^\dagger\\
B^\dagger & A_1 & B & C_2 & \cdots & C_{N-3} & C_{N-2}\\
C_2^\dagger & B^\dagger & A_2 & B & \cdots & C_{N-2} & C_{N-3}\\
\vdots  & \vdots  & \vdots  & \vdots & \ddots & \vdots & \vdots\\
C_{N-2}^\dagger & C_{N-3}^\dagger & C_{N-4}^\dagger & C_{N-5}^\dagger & \cdots & A_{N-2} & B\\
-B & C_{N-2}^\dagger & C_{N-3}^\dagger & C_{N-4}^\dagger & \cdots & B^\dagger & A_{N-1},
\end{pmatrix}
\end{equation}

where 

\begin{equation}
A_{j} = 
\begin{pmatrix}
-\mu f(j) & 0 \\
0 & +\mu f(j)
\end{pmatrix},
\end{equation}

\begin{equation}
B = \begin{pmatrix}
t/2 & -\Delta \\
+\Delta  & -t/2
\end{pmatrix},
\end{equation}

\begin{equation}
C_l = \begin{pmatrix}
0 & -\Delta/d_l^\alpha \\
+\Delta/d_l^\alpha  & 0
\end{pmatrix}.
\end{equation}

and $d_l = Min(l, N-l)$. In order to obtain these results, we have considered anti-periodic boundary conditions. This means that our Hamiltonian has the following form:
\begin{equation}
H = \sum_{n=0}^{N-2} \frac{t}{2} \left (c_n^\dagger c_{n+1} - c_{n+1} c_n^\dagger + c_{n+1}^\dagger c_n - c_{n} c_{n+1}^\dagger\right) -\frac{t}{2}\left(c_{N-1}^\dagger c_0 - c_0 c_{N-1}^\dagger + c_0^\dagger c_{N-1} - c_{N-1} c_0^\dagger\right) + \sum_{n=0}^{N-1} \mu f(n)\left (c^\dagger_{n}c_{n}-c_{n}c^\dagger_{n}\right) +\sum_{n=0}^{N-1}\sum_{l=1}^{N-1-n} \frac{\Delta}{d^\alpha_l}\left (c_{n+1}^\dagger c_n^\dagger - c_n^\dagger c_{n+l}^\dagger + c_n c_{n+l} - c_{n+l} c_{n}\right).
\end{equation}

### REAL SPACE HAMILTONIAN - SUPERCELLS

Now, let's try to express the same Hamiltonian using a different basis. Since the system has a periodicity of $q$ sites, we can split it into supercells. Let's consider that we have $L = N/q$ of such supercells, and let's consider the spinor $\chi_u^q = \left(c_{qu}, c^\dagger_{qu}, c_{qu+1}, c^\dagger_{qu+1}, ..., c_{qu+(q-1)}, c^\dagger_{qu+(q-1)}\right)^T$. The Hamiltonian can be expressed as follows:

\begin{equation}
    H = \sum_{u=0}^{L-1} \left[ (\chi_u^q)^\dagger H_{local} (\chi_u^q) + \left[(\chi_u^q)^\dagger H_{hop} (\chi_{u+1}^q) + h.c.\right] + \sum_{l=1}^{L-1}\left[(\chi_u^q)^\dagger H_{l} (\chi_{u+l}^q) + h.c.\right]\right].
\end{equation}
Again, we will need to impose anti-periodic boundary conditions and we obtain: 

\begin{equation}
H_{local} = 
\begin{pmatrix}
A_0 & B & C_2 & C_3 & \cdots & C_{q-2} & C_{q-1}\\
B^\dagger & A_1 & B & C_2 & \cdots & C_{q-3} & C_{q-2}\\
C_2^\dagger & B^\dagger & A_2 & B & \cdots & C_{q-2} & C_{q-3}\\
\vdots  & \vdots  & \vdots  & \vdots & \ddots & \vdots & \vdots\\
C_{q-2}^\dagger & C_{q-3}^\dagger & C_{q-4}^\dagger & C_{q-5}^\dagger & \cdots & A_{q-2} & B\\
C_{q-1}^\dagger & C_{q-2}^\dagger & C_{q-3}^\dagger & C_{q-4}^\dagger & \cdots & B^\dagger & A_{q-1}
\end{pmatrix},
\end{equation}

where 

\begin{equation}
A_{j} = 
\begin{pmatrix}
-\mu f(j) & 0 \\
0 & +\mu f(j)
\end{pmatrix},
\end{equation}

\begin{equation}
B = \begin{pmatrix}
t/2 & -\Delta \\
+\Delta  & -t/2
\end{pmatrix},
\end{equation}

\begin{equation}
C_l = \begin{pmatrix}
0 & -\Delta/d_l^\alpha \\
+\Delta/d_l^\alpha  & 0
\end{pmatrix}.
\end{equation}

and $d_l = Min(l, N-l)$. The other Hamiltonians are:

\begin{equation}
H_{PBC} = 
\begin{pmatrix}
0 & 0 & 0 & 0 &  \cdots & 0 & 0\\
0 & 0 & 0 & 0 & \cdots & 0 & 0\\
0 & 0 & 0 & 0 & \cdots & 0 & 0\\
\vdots  & \vdots  & \vdots  & \vdots & \ddots & \vdots & \vdots\\
0 & 0 & 0 & 0 & \cdots & 0 & 0\\
B & 0 & 0 & 0 & \cdots & 0 & 0
\end{pmatrix},
\end{equation}

where:

\begin{equation}
B = \begin{pmatrix}
t/2 & 0 \\
0  & -t/2.
\end{pmatrix}
\end{equation}

and:

\begin{equation}
H_{l} = 
\begin{pmatrix}
C_{l,0,0} & C_{l,0,1} & C_{l,0,2} & C_{l,0,3} & \cdots & C_{l,0,q-2} & C_{l,0,q-1}\\
C_{l,1,0} & C_{l,1,1} & C_{l,1,2} & C_{l,1,3} & \cdots & C_{l,1,q-2} & C_{l,1,q-1}\\
C_{l,2,0} & C_{l,2,1} & C_{l,2,2} & C_{l,2,3} & \cdots & C_{l,2,q-2} & C_{l,2,q-1}\\
\vdots  & \vdots  & \vdots  & \vdots & \ddots & \vdots & \vdots\\
C_{l,q-2,0} & C_{l,q-2,1} & C_{l,q-2,2} & C_{l,q-2,3} & \cdots & C_{l,q-2,q-2} & C_{l,q-2,q-1}\\
C_{l,q-1,0} & C_{l,q-1,1} & C_{l,q-1,2} & C_{l,q-1,3} & \cdots & C_{l,q-1,q-2} & C_{l,q-1,q-1}
\end{pmatrix},
\end{equation}

where:

\begin{equation}
C_{l,x,y} = \frac{\Delta}{2 d_{l,x,y}^\alpha}
\begin{pmatrix}
0 & -1 \\
1  & 0
\end{pmatrix}.
\end{equation}

and $d_{l,x,y} = Min(lq-(x-y), N-(lq-(x-y)))$. 

To obtain these results, we have imposed anti-periodic boundary conditions. This means that, for the local Hamiltonian we assume that $c_{uq+x+q} = -c_{uq+x}$. Then:

\begin{equation}
\sum_{u=0}^{L-1} (\chi_u^q)^\dagger H_{local} (\chi_u^q) = \sum_{u=0}^{L-1} \left[\sum_{x = 0}^{q-2} \frac{t}{2} \left (c_{uq+x}^\dagger c_{uq+x+1} - c_{uq+x+1} c_{uq+x}^\dagger + c_{uq+x+1}^\dagger c_{uq+x} - c_{uq+x} c_{uq+x+1}^\dagger\right) - \sum_{x=0}^{q-1} \mu f(uq+x)\left (c^\dagger_{uq+x}c_{uq+x}-c_{uq+x}c^\dagger_{uq+x}\right) + \sum_{x=0}^{q-1}\sum_{l=1}^{q-1-x} \frac{\Delta}{l^\alpha}\left (c_{uq+x+l}^\dagger c_{uq+x}^\dagger - c_{uq+x}^\dagger c_{uq+x+l}^\dagger + c_{uq+x} c_{uq+x+l} - c_{uq+x+l} c_{uq+x}\right)\right]
\end{equation}

For the Hamiltonians connecting different supercells, we have to use that $c_{(u+L)q + x} = -c_{uq+x}$. Then, we obtain:

\begin{equation}
\sum_{u=0}^{L-1} \left[ (\chi_u^q)^\dagger H_{hop} (\chi_{u+1}^q) + h.c. \right]= \sum_{u=0}^{L-2} \left[\frac{t}{2} \left (c_{uq+(q-1)}^\dagger c_{(u+1)q} - c_{(u+1)q} c_{uq+(q-1)}^\dagger + c_{(u+1)q}^\dagger c_{uq+(q-1)} - c_{uq+(q-1)} c_{(u+1)q}^\dagger\right)\right] +  \frac{t}{2} \left (c_{N-1}^\dagger c_{0} - c_{0} c_{N-1}^\dagger + c_{0}^\dagger c_{N-1} - c_{N-1} c_{0}^\dagger\right)
\end{equation}

\begin{equation}
\sum_{u=1}^{L-1} \sum_{l=1}^{L-1}\left[(\chi_u^q)^\dagger H_{l} (\chi_{u+l}^q) + h.c.\right] = \sum_{u=0}^{L-1} \sum_{l=1}^{L-1-u} \left[\sum_{x = 0}^{q-1}\sum_{x = 0}^{q-1} \frac{\Delta}{d_{l,x,y}^\alpha} \left (c_{(u+1)q+x}^\dagger c_{uq+y}^\dagger - c_{uq+y}^\dagger c_{(u+l)q+y}^\dagger + c_{uq+y} c_{(u+l)q+x} - c_{(u+l)q+x} c_{uq+y}\right)\right]
\end{equation}

which can also be expressed as:

\begin{equation}
\sum_{u=0}^{L-1} \left[\frac{t}{2} \left (c_{uq+(q-1)}^\dagger c_{(u+1)q} - c_{(u+1)q} c_{uq+(q-1)}^\dagger + c_{(u+1)q}^\dagger c_{uq+(q-1)} - c_{uq+(q-1)} c_{(u+1)q}^\dagger\right)\right]
\end{equation}

\begin{equation}
\sum_{u=0}^{L-1} \sum_{l=1}^{L-1} \left[\sum_{x = 0}^{q-1}\sum_{x = 0}^{q-1} \frac{\Delta}{2 d_{l,x,y}^\alpha} \left (c_{(u+1)q+x}^\dagger c_{uq+y}^\dagger - c_{uq+y}^\dagger c_{(u+l)q+y}^\dagger + c_{uq+y} c_{(u+l)q+x} - c_{(u+l)q+x} c_{uq+y}\right)\right]
\end{equation}

if we explicitly impose that $c_{(u+L)q + x} = -c_{uq+x}$. We will see in the next section how this is done.

### FOURIER TRANSFORM

Now, we want to write the Hamiltonian in momentum space. We transform the spinor $\chi_n^q$ as follows:
\begin{eqnarray}\label{eq:fourier}
    \chi_u^q = \frac{1}{\sqrt{L}} \sum_{k} e^{iku} \chi_k^q,
    (\chi_u^q)^\dagger = \frac{1}{\sqrt{L}} \sum_{k} e^{-iku} (\chi_k^q)^\dagger,
\end{eqnarray}
where $\chi_k^q= \left(c_{k,0}, c^\dagger_{-k,0}, c_{k, 1}, c^\dagger_{-k, 1}, ..., c_{k, q-1}, c^\dagger_{-k, q-1})\right)^T$. We need to find out which values of $k$ we are considering. Because of the anti-periodic boundary conditions, we want the following to hold:
\begin{equation}
    \chi_{u+L}^q = -\chi_{u}^q,
\end{equation}
therefore:
\begin{equation}
    \frac{1}{\sqrt{L}} \sum_{k} e^{ik(u+L)} \chi_k^q = -\frac{1}{\sqrt{L}} \sum_{k} e^{iku} \chi_k^q,
\end{equation}
meaning that $kL = (2m + 1)\pi$ where $m$ takes values from $\{0,1,2,...L-1\}$. 

Now, if we do the Fourier transform of the Hamiltonian we obtain:

\begin{equation}
    H = \sum_{k} \left[ (\chi_k^q)^\dagger H_{local} (\chi_k^q) + \left[e^{ik}(\chi_k^q)^\dagger H_{hop} (\chi_k^q) + h.c.\right] + \sum_{l=1}^{L-1}\left[e^{ikl}(\chi_k^q)^\dagger H_{l} (\chi_{k}^q) + h.c.\right]\right],
\end{equation}

where we have used:
\begin{equation}
\sum_{u=0}^{L-1} e^{i(k-k')u} = L\delta_{kk'}.
\end{equation}

### INFINITE SYSTEM

For the infinite system, $L$ goes to infinity. Then, we need to change $d_l = l$ and $d_{l,x,y} = lq-(x-y))$ because we are not using the anti-periodic boundary conditions. 
Then, the Hamiltonian in momentum space looks as follows:

\begin{equation}
    H = \sum_{k} \left[ (\chi_k^q)^\dagger H_{local} (\chi_k^q) + \left[e^{ik}(\chi_k^q)^\dagger H_{hop} (\chi_k^q) + h.c.\right] + (\chi_k^q)^\dagger H_{inf, l} (\chi_{k}^q) + h.c.\right],
\end{equation}

where:

\begin{equation}
H_{l, inf} = 
\begin{pmatrix}
C_{0,0} & C_{0,1} & C_{0,2} & C_{0,3} & \cdots & C_{0,q-2} & C_{0,q-1}\\
C_{1,0} & C_{1,1} & C_{1,2} & C_{1,3} & \cdots & C_{1,q-2} & C_{1,q-1}\\
C_{2,0} & C_{2,1} & C_{2,2} & C_{2,3} & \cdots & C_{2,q-2} & C_{2,q-1}\\
\vdots  & \vdots  & \vdots  & \vdots & \ddots & \vdots & \vdots\\
C_{q-2,0} & C_{q-2,1} & C_{q-2,2} & C_{q-2,3} & \cdots & C_{q-2,q-2} & C_{q-2,q-1}\\
C_{q-1,0} & C_{q-1,1} & C_{q-1,2} & C_{q-1,3} & \cdots & C_{q-1,q-2} & C_{q-1,q-1}
\end{pmatrix},
\end{equation}

\begin{equation}
C_{x,y} = f_{x,y}
\begin{pmatrix}
0 & -1 \\
1  & 0
\end{pmatrix}.
\end{equation}

where:

\begin{equation}
f_{x,y} = \sum_{l=1}^{\infty} \frac{\Delta e^{ikl}}{(lq-(x-y))^\alpha} = \frac{e^{ik}}{q^{\alpha}} \sum_{l=0}^{\infty} \frac{(e^{ik})^l}{(l+\frac{q+(x-y)}{q})^{\alpha}} = \frac{e^{ik}}{q^{\alpha}} HLP(e^{ik}, \alpha, \frac{q+(x-y)}{q})).
\end{equation}

HLP is the Hurwitz Lerth Phi function, also called the Lerch trascendent function.


## 1.1 REAL SPACE HAMILTONIAN WITH OBC

In [ ]:
#export

def H_OBC_Kitaev_LR_QP(params, mu, length, rot = False, AA = False):
    
    '''Compute the real space Hamiltonian with OBC for the Kitaev chain 
    with any value of alpha, Fn1/Fn and finite length'''

    alpha = params['alpha']
    t, delta = params['t'], params['delta']
    Fn1, Fn = params['Fn1'], params['Fn']
    phase, constant = params['phase'], params['constant']
    
    sx = np.array([[0, 1],[ 1, 0]])
    sy = np.array([[0, -1j],[1j, 0]])
    sz = np.array([[1, 0],[0, -1]])

    H_local = np.zeros((2*Fn*length, 2*Fn*length), dtype='complex')               
    B = t/2 * sz - delta * 1j * sy  
    for x in range(Fn*length):
        for y in np.arange(x, Fn*length):
            if x == y:
                if constant == True:
                    A_k = -mu * sz
                else:
                    if AA == False:
                        A_k = -mu * sz * (1-np.cos(2*np.pi*Fn1/Fn*x+phase))
                    else:
                        A_k = -mu * sz * (np.cos(2*np.pi*Fn1/Fn*x+phase))
                for j in [0,1]:
                    for l in [0,1]:
                        H_local[2*x+l][2*y+j] = A_k[l][j]
            elif y == x + 1:
                for j in [0,1]:
                    for l in [0,1]:
                        H_local[2*x+l][2*y+j] = B[l][j] 
                        H_local[2*y+l][2*x+j] = np.conjugate(B.T[l][j])
            else:
                d = abs(y-x)
                C = - delta/d**alpha * 1j * sy
                for j in [0,1]:
                    for l in [0,1]:
                        H_local[2*x+l][2*y+j] = C[l][j] 
                        H_local[2*y+l][2*x+j] = np.conjugate(C.T[l][j]) 
                        
    if rot == True:
        R = rot_sigma_y([0,1,0], np.pi/2, length*Fn)
        H_local = R@H_local@np.conjugate(R.T)
    
    return H_local


def H_OBC_Majoranas_Kitaev_LR_QP(params, mu, length, rot = False, AA = False):
    
    '''Compute the real space Hamiltonian with OBC for the Kitaev chain 
    with any value of alpha, Fn1/Fn and finite length. Here we use the Majorana basis.'''

    mu = mu*2
    alpha = params['alpha']
    t, delta = params['t']*2, params['delta']*4
    Fn1, Fn = params['Fn1'], params['Fn']
    phase, constant = params['phase'], params['constant']
    
    sx = np.array([[0, 1],[ 1, 0]])
    sy = np.array([[0, -1j],[1j, 0]])
    sz = np.array([[1, 0],[0, -1]])

    H_local = np.zeros((2*Fn*length, 2*Fn*length), dtype='complex')               
    B = t/4 * sy - delta/4 * 1j * sx  
    for x in range(Fn*length):
        for y in np.arange(x, Fn*length):
            if x == y:
                if constant == True:
                    A_k = -mu/2 * sy
                else:
                    if AA == False:
                        A_k = -mu/2 * sx * 1j * (1-np.cos(2*np.pi*Fn1/Fn*x+phase))
                    else:
                        A_k = -mu/2 * sy * (np.cos(2*np.pi*Fn1/Fn*x+phase))
                for j in [0,1]:
                    for l in [0,1]:
                        H_local[2*x+l][2*y+j] = A_k[l][j]
            elif y == x + 1:
                for j in [0,1]:
                    for l in [0,1]:
                        H_local[2*x+l][2*y+j] = B[l][j] 
                        H_local[2*y+l][2*x+j] = np.conjugate(B.T[l][j])
            else:
                d = abs(y-x)
                C = - delta/4/d**alpha * 1j * sx
                for j in [0,1]:
                    for l in [0,1]:
                        H_local[2*x+l][2*y+j] = C[l][j] 
                        H_local[2*y+l][2*x+j] = np.conjugate(C.T[l][j]) 
                        
    if rot == True:
        R = rot_sigma_y([0,1,0], np.pi/2, length*Fn)
        H_local = R@H_local@np.conjugate(R.T)
    
    return H_local

## 1.2 REAL SPACE HAMILTONIAN WITH APBC

In [ ]:
#export

def H_APBC_Kitaev_LR_QP(params, mu, length, rot=False, AA=False):

    '''Compute the real space Hamiltonian with APBC for the Kitaev chain 
    with any value of alpha, Fn1/Fn and finite length'''

    alpha = params['alpha']
    t, delta = params['t'], params['delta']
    Fn1, Fn = params['Fn1'], params['Fn']
    phase, constant = params['phase'], params['constant']
    
    sy = np.array([[0, -1j],[1j, 0]])
    sx = np.array([[0, 1],[ 1, 0]])
    sz = np.array([[1, 0],[0, -1]])
        
    H = np.zeros((2*length*Fn, 2*length*Fn), dtype='complex')               
    B = t/2 * sz - delta * 1j * sy  
    for x in range(Fn*length):
        for y in np.arange(x, Fn*length):
            if x == y:
                if constant == True:
                    A_k = -mu * sz
                else:
                    if AA == False:
                        A_k = -mu * sz * (1-np.cos(2*np.pi*Fn1/Fn*x+phase))
                    else:
                        A_k = -mu * sz * (np.cos(2*np.pi*Fn1/Fn*x+phase))
                for j in [0,1]:
                    for l in [0,1]:
                        H[2*x+l][2*y+j] = A_k[l][j]
            elif y == x + 1:
                for j in [0,1]:
                    for l in [0,1]:
                        H[2*x+l][2*y+j] = B[l][j] 
                        H[2*y+l][2*x+j] = np.conjugate(B.T[l][j])
            else:
                d = min(y-x, length*Fn-(y-x))
                C = - delta/d**alpha * 1j * sy
                for j in [0,1]:
                    for l in [0,1]:
                        H[2*x+l][2*y+j] = C[l][j] 
                        H[2*y+l][2*x+j] = np.conjugate(C.T[l][j]) 
   
    B = t/2 * sz 
    for j in [0,1]:
        for l in [0,1]:    
            H[2*(Fn*length-1)+l][j] += -B[l][j]
            H[l][2*(Fn*length-1)+j] += np.conjugate(-B.T[l][j]) 
            
    if rot == True:
        R = rot_sigma_y([0,1,0], np.pi/2, Fn*length)
        H = R@H@np.conjugate(R.T)

    return H

## 1.3 MOMENTUM SPACE HAMILTONIAN FOR FINITE LENGTH

In [ ]:
#export

def H_Kitaev_LR_QP(params, k, mu, length, rot=True, AA = False):
    
    '''Compute the momentum Hamiltonian with APBC for the Kitaev chain 
    with any value of alpha, Fn1/Fn and finite length'''
    
    alpha = params['alpha']
    t, delta = params['t'], params['delta']
    Fn1, Fn = params['Fn1'], params['Fn']
    phase, constant = params['phase'], params['constant']
    
    sx = np.array([[0, 1],[ 1, 0]])
    sy = np.array([[0, -1j],[1j, 0]])
    sz = np.array([[1, 0],[0, -1]])

    H_local = np.zeros((2*Fn, 2*Fn), dtype='complex')               
    B = t/2 * sz - delta * 1j * sy  
    for x in range(Fn):
        for y in np.arange(x, Fn):
            if x == y:
                if constant == True:
                    A_k = -mu * sz
                else:
                    if AA == False:
                        A_k = -mu * sz * (1-np.cos(2*np.pi*Fn1/Fn*x+phase))
                    else:
                        A_k = -mu * sz * (np.cos(2*np.pi*Fn1/Fn*x+phase))
                for j in [0,1]:
                    for l in [0,1]:
                        H_local[2*x+l][2*y+j] = A_k[l][j]
            elif y == x + 1:
                for j in [0,1]:
                    for l in [0,1]:
                        H_local[2*x+l][2*y+j] = B[l][j] 
                        H_local[2*y+l][2*x+j] = np.conjugate(B.T[l][j])
            else:
                d = min(abs(y-x), length*Fn-abs(y-x))
                C = - delta/d**alpha * 1j * sy
                for j in [0,1]:
                    for l in [0,1]:
                        H_local[2*x+l][2*y+j] = C[l][j] 
                        H_local[2*y+l][2*x+j] = np.conjugate(C.T[l][j]) 

    H_pbc_t = np.zeros((2*Fn, 2*Fn), dtype='complex') 
    B = t/2 * sz    
    for j in [0,1]:
        for l in [0,1]:    
            H_pbc_t[2*(Fn-1)+l][j] = B[l][j]*np.exp(1j*k)
            H_pbc_t[l][2*(Fn-1)+j] = np.conjugate(B.T[l][j])*np.exp(-1j*k)
            
 
    def block_matrix(Fn, k, delta, alpha, x, y, length):

        ''' Return block matrix C corresponding to the sum of H_l at position x, y'''

        C = np.zeros((2, 2), dtype='complex')  
        sy = np.array([[0, -1j],[1j, 0]])
        for l in range(1,length):
            d = min(l, length-l)
            d1 = min(Fn*l-(x-y), length*Fn-(Fn*l-(x-y)))
            d2 = min(Fn*l-(y-x), length*Fn-(Fn*l-(y-x)))
            C += - delta/2*1j*sy*(np.exp(1j*k*l)/d1**alpha - np.exp(-1j*k*l)/d2**alpha)

        return C
    

    H_pbc_delta = np.zeros((2*Fn, 2*Fn), dtype='complex')
    for x in range(Fn):
        for y in range(Fn):
            C = block_matrix(Fn, k, delta, alpha, x, y, length)
            for j in [0,1]:
                for l in [0,1]:
                     H_pbc_delta[2*x+l][2*y+j] = C[l][j]
    
    H_k = H_local + H_pbc_t + H_pbc_delta
    
    if rot == True:
        R = rot_sigma_y([0,1,0], np.pi/2, Fn)
        H_k = R@H_k@np.conjugate(R.T)

    return H_k

## 1.4 MOMENTUM SPACE HAMILTONIAN FOR INFINITE SYSTEM

In [ ]:
#export  

def H_Kitaev_LR_QP_inf(params, k, mu, rot=False, AA=True):
    
    '''Compute the momentum Hamiltonian with APBC for the Kitaev chain 
    with any value of alpha and Fn1/Fn. The system is infinite.'''
    
    alpha = params['alpha']
    t, delta = params['t'], params['delta']
    Fn1, Fn = params['Fn1'], params['Fn']
    phase, constant = params['phase'], params['constant']
    
    sx = np.array([[0, 1],[ 1, 0]])
    sy = np.array([[0, -1j],[1j, 0]])
    sz = np.array([[1, 0],[0, -1]])

    H_local = np.zeros((2*Fn, 2*Fn), dtype='complex')               
    B = t/2 * sz - delta * 1j * sy  
    for x in range(Fn):
        for y in np.arange(x, Fn):
            if x == y:
                if constant == True:
                    A_k = -mu * sz
                else:
                    if AA == False:
                        A_k = -mu * sz * (1-np.cos(2*np.pi*Fn1/Fn*x+phase))
                    else:
                        A_k = -mu * sz * (np.cos(2*np.pi*Fn1/Fn*x+phase))
                for j in [0,1]:
                    for l in [0,1]:
                        H_local[2*x+l][2*y+j] = A_k[l][j]
            elif y == x + 1:
                for j in [0,1]:
                    for l in [0,1]:
                        H_local[2*x+l][2*y+j] = B[l][j] 
                        H_local[2*y+l][2*x+j] = np.conjugate(B.T[l][j])
            else:
                d = y-x
                C = - delta/d**alpha * 1j * sy
                for j in [0,1]:
                    for l in [0,1]:
                        H_local[2*x+l][2*y+j] = C[l][j] 
                        H_local[2*y+l][2*x+j] = np.conjugate(C.T[l][j]) 

    H_pbc_t = np.zeros((2*Fn, 2*Fn), dtype='complex') 
    B = t/2 * sz    
    for j in [0,1]:
        for l in [0,1]:    
            H_pbc_t[2*(Fn-1)+l][j] = B[l][j]*np.exp(1j*k)
            H_pbc_t[l][2*(Fn-1)+j] = np.conjugate(B.T[l][j])*np.exp(-1j*k)
 
    H_pbc_delta = np.zeros((2*Fn, 2*Fn), dtype='complex')
    for x in range(Fn):
        for y in range(Fn):
            C = - delta * 1j * sy * np.exp(1j*k)/Fn**alpha*HLP(k, alpha, Fn, x, y)
            for j in [0,1]:
                for l in [0,1]:
                     H_pbc_delta[2*x+l][2*y+j] = C[l][j]
    
    H_k = H_local + H_pbc_t + H_pbc_delta + np.conjugate(H_pbc_delta.T)
    
    if rot == True:
        R = rot_sigma_y([0,1,0], np.pi/2, Fn)
        H_k = R@H_k@np.conjugate(R.T)
    
    return H_k


def d_k_H_Kitaev_LR_QP_inf(params, k, mu, rot=False, AA=True):
    
    ''' Analytical derivative of the Hamiltonian for k in momentum space for an infinite system. '''
    
    alpha = params['alpha']
    t, delta = params['t'], params['delta']
    Fn1, Fn = params['Fn1'], params['Fn']
    phase, constant = params['phase'], params['constant']
    
    sx = np.array([[0, 1],[ 1, 0]])
    sy = np.array([[0, -1j],[1j, 0]])
    sz = np.array([[1, 0],[0, -1]])

    d_H_pbc_t = np.zeros((2*Fn, 2*Fn), dtype='complex') 
    B = t/2 * sz    
    for j in [0,1]:
        for l in [0,1]:    
            d_H_pbc_t[2*(Fn-1)+l][j] = 1j * B[l][j]*np.exp(1j*k) 
            d_H_pbc_t[l][2*(Fn-1)+j] = -1j * np.conjugate(B.T[l][j])*np.exp(-1j*k)

    d_H_pbc_delta = np.zeros((2*Fn, 2*Fn), dtype='complex')
    for x in range(Fn):
        for y in range(Fn):
            C = - delta * 1j * sy * 1j * np.exp(1j*k)/Fn**(alpha+1) * (Fn*HLP(k, alpha-1, Fn, x, y)+(x-y)* HLP(k, alpha, Fn, x, y))
            for j in [0,1]:
                for l in [0,1]:
                     d_H_pbc_delta[2*x+l][2*y+j] = C[l][j]
    
    d_H_k = d_H_pbc_t + d_H_pbc_delta + np.conjugate(d_H_pbc_delta.T)
    
    if rot == True:
        R = rot_sigma_y(np.array([0,1,0]), np.pi/2, Fn)
        d_H_k = R@d_H_k@np.conjugate(R.T)
    
    return d_H_k
    
    
def d_phase_H_Kitaev_LR_QP_inf(params, k, mu, rot=False, AA=True):
    
    ''' Analytical derivative of the Hamiltonian for phi in momentum space for an infinite system. '''

    alpha = params['alpha']
    t, delta = params['t'], params['delta']
    Fn1, Fn = params['Fn1'], params['Fn']
    phase, constant = params['phase'], params['constant']
    
    sx = np.array([[0, 1],[ 1, 0]])
    sy = np.array([[0, -1j],[1j, 0]])
    sz = np.array([[1, 0],[0, -1]])

    d_H_local = np.zeros((2*Fn, 2*Fn), dtype='complex')               
    for x in range(Fn):
        for y in np.arange(x, Fn):
            if x == y:
                if constant == True:
                    A_k = 0
                else:
                    if AA == False:
                        A_k = -mu * sz * (np.sin(2*np.pi*Fn1/Fn*x+phase))
                    else:
                        A_k = mu * sz * (np.sin(2*np.pi*Fn1/Fn*x+phase))
                for j in [0,1]:
                    for l in [0,1]:
                        d_H_local[2*x+l][2*y+j] = A_k[l][j]
    
    d_H_k = d_H_local
    
    if rot == True:
        R = rot_sigma_y(np.array([0,1,0]), np.pi/2, Fn)
        d_H_k = R@d_H_k@np.conjugate(R.T)
    
    return d_H_k

## 1.5 UPPER-RIGHT BLOCK OF THE CHIRAL HAMILTONIAN - FINITE SYSTEM

In [ ]:
#export

def h_chiral_Kitaev_LR_QP(params, k, mu, length, AA=False):
    
    '''Computes one block of the off-diagonal Hamiltonian in momentum space for a finite system with APBC'''
    
    alpha = params['alpha']
    t, delta = params['t'], params['delta']
    Fn1, Fn = params['Fn1'], params['Fn']
    phase, constant = params['phase'], params['constant']

    H_local = np.zeros((Fn, Fn), dtype='complex')                
    for x in range(Fn):
        for y in np.arange(x, Fn):
            if x == y:
                if constant == True:
                    H_local[x][y] = -mu 
                else:
                    if AA == False:
                        H_local[x][y] = -mu * (1-np.cos(2*np.pi*Fn1/Fn*x+phase))
                    else:
                        H_local[x][y] = -mu * (np.cos(2*np.pi*Fn1/Fn*x+phase))
            elif y == x + 1:
                    H_local[x][y] = -delta + t/2
                    H_local[y][x] = + delta + t/2
            else:
                d = min(abs(y-x), length*Fn-abs(y-x))
                H_local[x][y] = -delta/d**alpha 
                H_local[y][x] = + delta/d**alpha

    H_pbc_t = np.zeros((Fn, Fn), dtype='complex') 
    H_pbc_t[Fn-1][0] = t/2*np.exp(1j*k)
    H_pbc_t[0][Fn-1] = t/2*np.exp(-1j*k)
 
    H_pbc_delta = np.zeros((Fn, Fn), dtype='complex')
    for x in range(Fn):
        for y in range(Fn):
            c = 0
            for l in range(1,length):
                d1 = min(Fn*l-(x-y), length*Fn-(Fn*l-(x-y)))
                d2 = min(Fn*l-(y-x), length*Fn-(Fn*l-(y-x)))
                c += - delta/2*(np.exp(1j*k*l)/d1**alpha - np.exp(-1j*k*l)/d2**alpha)
            H_pbc_delta[x][y] = c
    
    Hc = H_local + H_pbc_t + H_pbc_delta

    return Hc

## 1.5 UPPER-RIGHT BLOCK OF THE CHIRAL HAMILTONIAN - INFINITE SYSTEM

** I separated HLP_d and the infinite analytical expressions for the Hamiltonian in order to make the code more efficient

In [ ]:
#export

def h_chiral_Kitaev_LR_QP_inf(params, k, mu, AA, H_pbc_delta):
    
    '''Computes one block of the off-diagonal Hamiltonian in momentum space for an infinite system'''
    
    alpha = params['alpha']
    t, delta = params['t'], params['delta']
    Fn1, Fn = params['Fn1'], params['Fn']
    phase, constant = params['phase'], params['constant']

    H_local = np.zeros((Fn, Fn), dtype='complex')                
    for x in range(Fn):
        for y in np.arange(x, Fn):
            if x == y:
                if constant == True:
                    H_local[x][y] = -mu 
                else:
                    if AA == False:
                        H_local[x][y] = -mu * (1-np.cos(2*np.pi*Fn1/Fn*x+phase))
                    else:
                        H_local[x][y] = -mu * (np.cos(2*np.pi*Fn1/Fn*x+phase))
            elif y == x + 1:
                    H_local[x][y] = -delta + t/2
                    H_local[y][x] = + delta + t/2
            else:
                d = (y-x)
                H_local[x][y] = -delta/d**alpha 
                H_local[y][x] = + delta/d**alpha

    H_pbc_t = np.zeros((Fn, Fn), dtype='complex') 
    H_pbc_t[Fn-1][0] = t/2*np.exp(1j*k)
    H_pbc_t[0][Fn-1] = t/2*np.exp(-1j*k)
    
    Hc = H_local + H_pbc_t + H_pbc_delta - np.conjugate(H_pbc_delta.T)

    return Hc

def H_pbc_sp(params, k):
    
    '''Computes the analytical expression of the long-range infinite part of 
    the off-diagonal Hamiltonian. Does not depend on the chemical potential.'''
    
    alpha = params['alpha']
    delta = params['delta']
    Fn1, Fn = params['Fn1'], params['Fn']
       
    H_pbc_delta = np.zeros((Fn, Fn), dtype='complex')
    for dist in np.arange(-(Fn-1), Fn):
        c = - delta * np.exp(1j*k)/Fn**alpha * HLP_d(k, alpha, Fn, dist)
        for x in range(Fn):
            for y in range(Fn):
                if dist == (y-x):
                    H_pbc_delta[x][y] = c

    return H_pbc_delta

def d_h_chiral_Kitaev_LR_QP_inf(params, k, mu, AA, d_H_pbc_delta):
    
    '''Computes the analytical derivative of k of one block of the off-diagonal Hamiltonian 
    in momentum space for an infinite system'''
    
    alpha = params['alpha']
    t, delta = params['t'], params['delta']
    Fn1, Fn = params['Fn1'], params['Fn']
    phase, constant = params['phase'], params['constant']

    d_H_pbc_t = np.zeros((Fn, Fn), dtype='complex') 
    d_H_pbc_t[Fn-1][0] = 1j*t/2*np.exp(1j*k)
    d_H_pbc_t[0][Fn-1] = -1j*t/2*np.exp(-1j*k)
    
    Hc = d_H_pbc_t + d_H_pbc_delta - np.conjugate(d_H_pbc_delta.T)

    return Hc

def d_H_pbc_sp(params, k):
    
    '''Computes the analytical derivative of the long-range infinite part of 
    the off-diagonal Hamiltonian. Does not depend on the chemical potential.'''
    
    alpha = params['alpha']
    delta = params['delta']
    Fn1, Fn = params['Fn1'], params['Fn']
    
    d_H_pbc_delta = np.zeros((Fn, Fn), dtype='complex')
    for dist in np.arange(-(Fn-1), Fn):
        c = - delta * 1j * np.exp(1j*k)/Fn**(alpha+1) * (Fn*HLP_d(k, alpha-1, Fn, dist)- dist * HLP_d(k, alpha, Fn, dist)) 
        for x in range(Fn):
            for y in range(Fn):
                if dist == (y-x):
                    d_H_pbc_delta[x][y] = c

    return d_H_pbc_delta

def HLP_d(k, alpha, Fn, dist):
    
    '''Closed form of sum of exponens using HLP function. dist is y-x'''
    
    f = lerchphi(np.exp(1j*k),alpha,(Fn+(dist))/Fn)
    return np.float(re(f))+ 1j*np.float(im(f)) 

# 2. LONG RANGE TWO-BAND SYSTEM


In order to obtain the Hamiltonian in the momentum space, we start from the follwing Hamiltonian:

\begin{equation}\label{eq: hamiltonian}
    H = \sum_{n=0}^{N-1} \left [t \left (c^\dagger_{n+1}c_{n} + c^\dagger_{n}c_{n+1}\right) - \mu \left ( 2c^\dagger_{n}c_{n}-1\right) + \sum_{l=1}^{N-1}\frac{\Delta}{d^\alpha_l}\left (c^\dagger_{n+l}c^\dagger_{n} + c_{n}c_{n+l}\right)\right],
\end{equation}

where $t$ is the hopping amplitude, $\mu$ is the chemical potential, $\Delta$ is the superconducting pairing amplitude and $c_{n}(c^\dagger_{n})$ are the annihilation (creation) operators at the $n$th site of the chain. Consider we have a $N$ sites in the chain and our Hamiltonian is periodic with one site per unit cell.

## CALCULATIONS

### REAL SPACE HAMILTONIAN

Let's consider the following basis $\chi = \left(c_{0}, c^\dagger_{0}, c_{1}, c^\dagger_{1}, ..., c_{N-1}, c^\dagger_{N-1}\right)^T$. The Hamiltonian can be expressed as follows:

\begin{equation}
    H = \chi^\dagger H_N \chi
\end{equation}
where:
\begin{equation}
H_N = 
\begin{pmatrix}
A_0 & B & C_2 & C_3 & \cdots & C_{N-2} & -B^\dagger\\
B^\dagger & A_1 & B & C_2 & \cdots & C_{N-3} & C_{N-2}\\
C_2^\dagger & B^\dagger & A_2 & B & \cdots & C_{N-2} & C_{N-3}\\
\vdots  & \vdots  & \vdots  & \vdots & \ddots & \vdots & \vdots\\
C_{N-2}^\dagger & C_{N-3}^\dagger & C_{N-4}^\dagger & C_{N-5}^\dagger & \cdots & A_{N-2} & B\\
-B & C_{N-2}^\dagger & C_{N-3}^\dagger & C_{N-4}^\dagger & \cdots & B^\dagger & A_{N-1},
\end{pmatrix}
\end{equation}

where 

\begin{equation}
A_{j} = 
\begin{pmatrix}
-\mu & 0 \\
0 & +\mu
\end{pmatrix},
\end{equation}

\begin{equation}
B = \begin{pmatrix}
t/2 & -\Delta \\
+\Delta  & -t/2
\end{pmatrix},
\end{equation}

\begin{equation}
C_l = \begin{pmatrix}
0 & -\Delta/d_l^\alpha \\
+\Delta/d_l^\alpha  & 0
\end{pmatrix}.
\end{equation}

and $d_l = Min(l, N-l)$. In order to obtain these results, we have considered anti-periodic boundary conditions. This means that our Hamiltonian has the following form:
\begin{equation}
H = \sum_{n=0}^{N-2} \frac{t}{2} \left (c_n^\dagger c_{n+1} - c_{n+1} c_n^\dagger + c_{n+1}^\dagger c_n - c_{n} c_{n+1}^\dagger\right) -\frac{t}{2}\left(c_{N-1}^\dagger c_0 - c_0 c_{N-1}^\dagger + c_0^\dagger c_{N-1} - c_{N-1} c_0^\dagger\right) + \sum_{n=0}^{N-1} \mu f(n)\left (c^\dagger_{n}c_{n}-c_{n}c^\dagger_{n}\right) +\sum_{n=0}^{N-1}\sum_{l=1}^{N-1-n} \frac{\Delta}{d^\alpha_l}\left (c_{n+1}^\dagger c_n^\dagger - c_n^\dagger c_{n+l}^\dagger + c_n c_{n+l} - c_{n+l} c_{n}\right).
\end{equation}

### REAL SPACE HAMILTONIAN - SUPERCELLS

Now, let's try to express the same Hamiltonian using a different basis. Since the system has a periodicity of a site, we can split it into unit cells. Let's consider the spinor $\chi_u = \left(c_{u}, c^\dagger_{u}\right)^T$. The Hamiltonian can be expressed as follows:

\begin{equation}
    H = \sum_{u=0}^{N-1} \left[ (\chi_u^q)^\dagger H_{local} (\chi_u^q) + \left[(\chi_u^q)^\dagger H_{hop} (\chi_{u+1}^q) + h.c.\right] + \sum_{l=1}^{L-1}\left[(\chi_u^q)^\dagger H_{l} (\chi_{u+l}^q) + h.c.\right]\right].
\end{equation}
Again, we will need to impose anti-periodic boundary conditions and we obtain: 

\begin{equation}
H_{local} = 
\begin{pmatrix}
-\mu  & 0 \\
0 & +\mu 
\end{pmatrix},
\end{equation}

\begin{equation}
H_{PBC}= \begin{pmatrix}
t/2 & 0 \\
0  & -t/2
\end{pmatrix},
\end{equation}

\begin{equation}
C_l = \begin{pmatrix}
0 & -\Delta/d_l^\alpha \\
+\Delta/d_l^\alpha  & 0
\end{pmatrix}.
\end{equation}

and $d_l = Min(l, N-l)$. 

Here we explicitly impose that $c_{u+N} = -c_{u}$.

### FOURIER TRANSFORM

Now, we want to write the Hamiltonian in momentum space. We transform the spinor $\chi_u$ as follows:
\begin{eqnarray}
    \chi_u = \frac{1}{\sqrt{N}} \sum_{k} e^{iku} \chi_k,\\
    (\chi_u)^\dagger = \frac{1}{\sqrt{N}} \sum_{k} e^{-iku} (\chi_k)^\dagger,
\end{eqnarray}
where $\chi_k= \left(c_{k}, c^\dagger_{-k}\right)^T$. We need to find out which values of $k$ we are considering. Because of the anti-periodic boundary conditions, we want the following to hold:
\begin{equation}
    \chi_{u+N} = -\chi_{u},
\end{equation}
therefore:
\begin{equation}
    \frac{1}{\sqrt{N}} \sum_{k} e^{ik(u+N)} \chi_k = -\frac{1}{\sqrt{N}} \sum_{k} e^{iku} \chi_k,
\end{equation}
meaning that $kN = (2m + 1)\pi$ where $m$ takes values from $\{0,1,2,...N-1\}$. 

Now, if we do the Fourier transform of the Hamiltonian we obtain:

\begin{equation}
    H = \sum_{k} \left[ (\chi_k)^\dagger H_{local} (\chi_k) + \left[e^{ik}(\chi_k)^\dagger H_{hop} (\chi_k) + h.c.\right] + \sum_{l=1}^{N-1}\left[e^{ikl}(\chi_k)^\dagger H_{l} (\chi_{k}) + h.c.\right]\right],
\end{equation}

where we have used:
\begin{equation}
\sum_{u=0}^{L-1} e^{i(k-k')u} = L\delta_{kk'}.
\end{equation}

we can simplify this in the following form:

\begin{equation}
    H = \sum_{k} (\chi_k)^\dagger \left[ H_{local}  + 2H_{hop}\cos(k) + \sum_{l=1}^{N-1}\left(H_{l}\sin(kl)\right)\right]\chi_k,
\end{equation}

### INFINITE SYSTEM
When $N \rightarrow \infty$, we can use the following identity:
\begin{equation}
\sum_{l=1}^{N-1}\Delta\frac{\sin(kl)}{d_l^\alpha} = -\frac{\Delta i}{2}(polylog(\alpha,e^{ik})-polylog(\alpha,e^{-ik})),
\end{equation}
assuming that in this case we don't need to apply the periodic boundary conditions, and thus $d_l = l$.

## 2.1 REAL SPACE OBC HAMILTONIAN

In [ ]:
#export

def H_OBC_Kitaev_twoband(params, mu, length, rot=False):
    
    '''Compute the real space Hamiltonian with APBC for the Kitaev chain 
    with any value of alpha and finite length'''

    alpha = params['alpha']
    t, delta = params['t'], params['delta']
    
    sy = np.array([[0, -1j],[1j, 0]])
    sz = np.array([[0, 1],[ 1, 0]])
    sx = np.array([[1, 0],[0, -1]])
        
    H = np.zeros((2*length, 2*length), dtype='complex')               
    B = t/2 * sz - delta * 1j * sy  
    for x in range(length):
        for y in np.arange(x, length):
            if x == y:
                A_k = -mu * sz
                for j in [0,1]:
                    for l in [0,1]:
                        H[2*x+l][2*y+j] = A_k[l][j]
            elif y == x + 1:
                for j in [0,1]:
                    for l in [0,1]:
                        H[2*x+l][2*y+j] = B[l][j] 
                        H[2*y+l][2*x+j] = np.conjugate(B.T[l][j])
            else:
                d = y-x
                C = - delta/d**alpha * 1j * sy
                for j in [0,1]:
                    for l in [0,1]:
                        H[2*x+l][2*y+j] = C[l][j] 
                        H[2*y+l][2*x+j] = np.conjugate(C.T[l][j]) 
                        
    if rot == True:
        R = rot_sigma_y([0,1,0], np.pi/2, length)
        H = R@H@np.conjugate(R.T)

    return H

## 2.2 REAL SPACE APBC HAMILTONIAN

In [ ]:
#export

def H_APBC_Kitaev_twoband(params, mu, length, rot=False):
    
    '''Compute the real space Hamiltonian with APBC for the Kitaev chain 
    with any value of alpha and finite length'''

    alpha = params['alpha']
    t, delta = params['t'], params['delta']
    
    sy = np.array([[0, -1j],[1j, 0]])
    sz = np.array([[0, 1],[ 1, 0]])
    sx = np.array([[1, 0],[0, -1]])
        
    H = np.zeros((2*length, 2*length), dtype='complex')               
    B = t/2 * sz - delta * 1j * sy  
    for x in range(length):
        for y in np.arange(x, length):
            if x == y:
                A_k = -mu * sz
                for j in [0,1]:
                    for l in [0,1]:
                        H[2*x+l][2*y+j] = A_k[l][j]
            elif y == x + 1:
                for j in [0,1]:
                    for l in [0,1]:
                        H[2*x+l][2*y+j] = B[l][j] 
                        H[2*y+l][2*x+j] = np.conjugate(B.T[l][j])
            else:
                d = min(y-x, length-(y-x))
                C = - delta/d**alpha * 1j * sy
                for j in [0,1]:
                    for l in [0,1]:
                        H[2*x+l][2*y+j] = C[l][j] 
                        H[2*y+l][2*x+j] = np.conjugate(C.T[l][j]) 
   
    B = t/2 * sz 
    for j in [0,1]:
        for l in [0,1]:    
            H[2*(length-1)+l][j] += -B[l][j]
            H[l][2*(length-1)+j] += np.conjugate(-B.T[l][j]) 
            
    if rot == True:
        R = rot_sigma_y([0,1,0], np.pi/2, length)
        H = R@H@np.conjugate(R.T)

    return H

## 2.3 MOMENTUM SPACE FINITE SYSTEM

In [ ]:
#export

def H_Kitaev_twoband(params, k, mu, length, rot=True):
    
    '''Compute the two band momentum Hamiltonian with APBC for the Kitaev chain 
    with any value of alpha and finite length'''
    
    alpha = params['alpha']
    t, delta = params['t'], params['delta']
    
    sy = np.array([[0, -1j],[1j, 0]])
    if rot == True:
        sz = np.array([[0, 1],[ 1, 0]])
        sx = np.array([[1, 0],[0, -1]])        
    else:
        sx = np.array([[0, 1],[ 1, 0]])
        sz = np.array([[1, 0],[0, -1]])
        
    H_local = -mu * sz           
    H_hop = t * sz * np.cos(k)
    
    H_l = np.zeros((2,2), dtype='complex')
    for l in range(1,length):
        d = min(l, length-l)
        H_l += -delta/d**alpha*sy*np.sin(k*d)
    
    H_k = H_local + H_hop + H_l
    
    return H_k

## 2.4 MOMENTUN SPACE INFINITE SYSTEM

In [ ]:
#export

def H_Kitaev_twoband_inf(params, k, mu, rot=True):
    
    '''Compute the bulk Hamiltonian for the infinite Kitaev chain'''

    alpha = params['alpha']
    t, delta = params['t'], params['delta']*2
    
    sx = np.array([[0, 1],[ 1, 0]])
    sy = np.array([[0, -1j],[1j, 0]])
    sz = np.array([[1, 0],[0, -1]])

    if rot == False:
        d = np.array([0, -delta*f(k, alpha), t*np.cos(k)-mu])
    else:
        d = np.array([t*np.cos(k)-mu, -delta*f(k, alpha), 0])
    H = d[0]*sx + d[1]*sy + d[2]*sz

    return H


def analytic_Kitaev_twoband(params, k, mu, rot=True):
    
    '''Compute the analytic form of the evals and evecs for the 
    bulk Hamiltonian for the two-band infinite Kitaev chain'''

    alpha = params['alpha']
    t, delta = params['t'], params['delta']
    
    sx = np.array([[0, 1],[ 1, 0]])
    sy = np.array([[0, -1j],[1j, 0]])
    sz = np.array([[1, 0],[0, -1]])

    if rot == False:
        d = np.array([0, delta*f(k, alpha), t*np.cos(k)-mu])
    else:
        d = np.array([t*np.cos(k)-mu, delta*f(k, alpha), 0])
    
    evec1 = np.array([d[2]+np.linalg.norm(d), d[0]+1j*d[1]])
    evec2 = np.array([1j*d[1]-d[0], d[2]+np.linalg.norm(d)])
    evecs = np.array([evec1, evec2]).T
    
    E = np.linalg.norm(d)
    evals = np.array([-E,E])    
    
    return evals, evecs

## 2.5 UPPER-RIGHT BLOCK OF THE CHIRAL HAMILTONIAN - FINITE SYSTEM

In [ ]:
#export

def h_Kitaev_twoband(params, k, mu, length):
    
    '''Compute one block of the off-diagonal form of two band momentum Hamiltonian 
    with APBC for the Kitaev chain. Finite system.'''
    
    alpha = params['alpha']
    t, delta = params['t'], params['delta']
    
    f = 0
    for l in range(1,length):
        d = min(l, length-l)
        f += 1/d**alpha*np.sin(k*d)
    h = -mu + t * np.cos(k) - 1j*delta*f
    
    return h


def d_h_Kitaev_twoband(params, k, mu, length, rotation=True):
    
    '''Compute the analytical derivative of one block of the off-diagonal form of two band 
    momentum Hamiltonian with APBC for the Kitaev chain. Finite system. There is a problem 
    with the definition of the derivative.'''
    
    alpha = params['alpha']
    t, delta = params['t'], params['delta']
    
    df = 0
    for l in range(1,length):
        d = min(l, length-l)
        l_mod = d
        df += 1/d**alpha*l_mod*np.cos(k*l_mod)
    dh = - t * np.sin(k) - 1j*delta*df
    
    return dh

## 2.6 UPPER-RIGHT BLOCK OF THE CHIRAL HAMILTONIAN - INFINITE SYSTEM

In [ ]:
#export

def h_Kitaev_twoband_inf(params, k, mu, rotation=True):
    
    '''Compute one block of the off-diagonal form of two band momentum Hamiltonian 
    with APBC for the Kitaev chain. Infinite system.'''

    alpha = params['alpha']
    t, delta = params['t'], params['delta']
    h = t*np.cos(k)-mu - 1j*delta*f(k, alpha)

    return h


def d_h_Kitaev_twoband_inf(params, k, mu, rotation=True):
    
    '''Compute the analytical derivative of one block of the off-diagonal form of two band 
    momentum Hamiltonian with APBC for the Kitaev chain. Infinite system.'''
    
    alpha = params['alpha']
    t, delta = params['t'], params['delta']
    d_h = - t*np.sin(k) - 1j*delta*df(k, alpha)

    return d_h

# 3. QP SHORT RANGE HAMILTONIANS

In order to obtain the Hamiltonian in the momentum space, we start from the follwing Hamiltonian:

\begin{equation}\label{eq: hamiltonian}
    H = \sum_{n=1}^{N} \left [\frac{t}{2} \left (c^\dagger_{n+1}c_{n} + c^\dagger_{n}c_{n+1}\right) - \mu f(n) c^\dagger_{n}c_{n} + \frac{\Delta}{2}\left (c^\dagger_{n+1}c^\dagger_{n} + c^\dagger_{n}c^\dagger_{n+1} \right)\right] + h.c.,
\end{equation}

where $t$ is the hopping amplitude, $\mu$ is the chemical potential, $\Delta$ is the superconducting pairing amplitude and $c_{n}(c^\dagger_{n})$ are the annihilation (creation) operators at the $n$th site of the chain.

Now, we will consider $f(i) = \cos\left (2 \pi i \frac{F_{n-1}}{F_{n}}\right )$, where $F_{n-1},F_n$ are integer from the Fibonacci sequence. Therefore, our Hamiltonian has a periodicity of $F_n = q$ sites. In the thermodynamic limit, que can express it in terms of $H_local$, which defines the supercell, and $H_{PBC}$ which defines the periodic boundary conditions.

Let's consider the spinor $C_n^q = \left(c_{qn}, c^\dagger_{qn}, c_{qn+1}, c^\dagger_{qn+1}, ..., c_{qn+q-1}, c^\dagger_{qn+q-1}\right)^T$, which is a basis for the $n$th supercell of the system. The Hamiltonian can be expressed as follows:

\begin{equation}
    H = \sum_n \left[(C_n^q)^\dagger H_{local}^q (C_n^q) + [(C_n^q)^\dagger H_{PBC}^q (C_{n+1}^q) + H.c.]\right],
\end{equation}
where:

\begin{equation}
H_{local} = 
\begin{pmatrix}
A_0 & B & 0 & 0 &  \cdots & 0 & 0\\
B^\dagger & A_1 & B & 0 & \cdots & 0 & 0\\
0 & B^\dagger & A_2 & B & \cdots & 0 & 0\\
\vdots  & \vdots  & \vdots  & \vdots & \ddots & \vdots & \vdots\\
0 & 0 & 0 & 0 & \cdots & A_{q-2} & B\\
0 & 0 & 0 & 0 & \cdots & B^\dagger & A_{q-1},
\end{pmatrix}
\end{equation}

\begin{equation}
H_{PBC} = 
\begin{pmatrix}
0 & 0 & 0 & 0 &  \cdots & 0 & 0\\
0 & 0 & 0 & 0 & \cdots & 0 & 0\\
0 & 0 & 0 & 0 & \cdots & 0 & 0\\
\vdots  & \vdots  & \vdots  & \vdots & \ddots & \vdots & \vdots\\
0 & 0 & 0 & 0 & \cdots & 0 & 0\\
B & 0 & 0 & 0 & \cdots & 0 & 0,
\end{pmatrix}
\end{equation}

and:

\begin{equation}
A_{j} = 
\begin{pmatrix}
-\mu\cos(\frac{2\pi j}{q}) & 0 \\
0 & +\mu\cos(\frac{2\pi j}{q}),
\end{pmatrix}
\end{equation}

\begin{equation}
B = \frac{1}{2}
\begin{pmatrix}
-t & -\Delta \\
+\Delta  & t.
\end{pmatrix}
\end{equation}

Now, we want to write the Hamiltonian in momentum space. In the thermodynamic limit, we can transform the spinor $C_n^q$ as follows:
\begin{equation}
    C_n^q = \frac{1}{2\pi}\int_{[-\pi,\pi]} dk e^{ikn}C_k^q,
\end{equation}
where $C_k^q = \left(c_{k,0}, c^\dagger_{-k,0}, c_{k, 1}, c^\dagger_{-k, 1}, ..., c_{k, q-1}, c^\dagger_{-k, q-1})\right)^T$, using:

\begin{equation}
\sum_{n=1}^N e^{i2\pi(k-k')n} = N\delta_{kk'}
\end{equation}

we obtain that:

\begin{equation}
    H = \frac{1}{2\pi}\int_{[-\pi,\pi]} dk (C_k^q)^\dagger\left[H_{local} + (e^{ik}H_{PBC} + H.c) \right](C_k^q),
\end{equation}

therefore:

\begin{equation}
H_k = H_{local} + (e^{ik}H_{PBC} + H.c) 
\end{equation}

## 3.1 MOMENTUM SPACE HAMILTONIAN FOR INFINITE SYSTEM

In [ ]:
#export

def H_OBC_Kitaev_SR_QP(params, mu, length, rot=False, AA=False):
    
    '''Compute the real space OBC Hamiltonian for the Kitaev chain 
    with any value of alpha and Fn1/Fn and short range'''
    
    t, delta = params['t'], params['delta']
    Fn1, Fn = params['Fn1'], params['Fn']
    phase, constant = params['phase'], params['constant']
    sx = np.array([[0, 1],[ 1, 0]])
    sy = np.array([[0, -1j],[1j, 0]])
    sz = np.array([[1, 0],[0, -1]])

    H_local = np.zeros((2*Fn*length, 2*Fn*length), dtype='complex')
    
    B = t/2 * sz + delta/2 * 1j * sy   
    for i in range(Fn*length-1):
        if constant == True:
            A_k = mu * sz
        else:
            if AA == False:
                A_k = mu * sz * (1-np.cos(2*np.pi*Fn1/Fn*i+phase))
            else:
                A_k = mu * sz * (np.cos(2*np.pi*Fn1/Fn*i+phase))
        for j in [0,1]:
            for l in [0,1]:
                H_local[2*i+j][2*i+l] = A_k[j][l]
                H_local[2*i+l][2*(i+1)+j] = B[j][l] 
                H_local[2*(i+1)+j][2*i+l] = np.conjugate(B.T[l][j]) 

    i = Fn*length-1
    if constant == True:
        A_k = mu * sz
    else:
        if AA == False:
            A_k = mu * sz * (1-np.cos(2*np.pi*Fn1/Fn*i+phase))
        else:
            A_k = mu * sz * (np.cos(2*np.pi*Fn1/Fn*i+phase))
    for j in [0,1]:
        for l in [0,1]:
            H_local[2*i+j][2*i+l] = A_k[j][l]  
            
    if rot == True:
        R = rot_sigma_y([0,1,0], np.pi/2, length*Fn)
        H_local = R@H_local@np.conjugate(R.T)
  
    return H_local


def H_OBC_Majoranas_Kitaev_SR_QP(params, mu, length, rot = False, AA = False):
    
    '''Compute the real space Hamiltonian with OBC for the Kitaev chain 
    with any value of alpha, Fn1/Fn and finite length. Here we use the Majorana basis.'''

    mu = mu*2
    alpha = params['alpha']
    t, delta = params['t']*2, params['delta']*4
    Fn1, Fn = params['Fn1'], params['Fn']
    phase, constant = params['phase'], params['constant']
    
    sx = np.array([[0, 1],[ 1, 0]])
    sy = np.array([[0, -1j],[1j, 0]])
    sz = np.array([[1, 0],[0, -1]])

    H_local = np.zeros((2*Fn*length, 2*Fn*length), dtype='complex')               
    B = t/4 * sy - delta/4 * 1j * sx  
    for x in range(Fn*length):
        for y in np.arange(x, Fn*length):
            if x == y:
                if constant == True:
                    A_k = -mu/2 * sy
                else:
                    if AA == False:
                        A_k = -mu/2 * sx * 1j * (1-np.cos(2*np.pi*Fn1/Fn*x+phase))
                    else:
                        A_k = -mu/2 * sy * (np.cos(2*np.pi*Fn1/Fn*x+phase))
                for j in [0,1]:
                    for l in [0,1]:
                        H_local[2*x+l][2*y+j] = A_k[l][j]
            elif y == x + 1:
                for j in [0,1]:
                    for l in [0,1]:
                        H_local[2*x+l][2*y+j] = B[l][j] 
                        H_local[2*y+l][2*x+j] = np.conjugate(B.T[l][j])
                        
    if rot == True:
        R = rot_sigma_y([0,1,0], np.pi/2, length*Fn)
        H_local = R@H_local@np.conjugate(R.T)
    
    return H_local


def H_Kitaev_SR_QP(params, k, mu, rot=False, AA=False):
    
    '''Compute the bulk Hamiltonian (Nambu spinor basis) for the Kitaev chain 
    with any value of alpha and Fn1/Fn'''
    
    t, delta = params['t'], params['delta']
    Fn1, Fn = params['Fn1'], params['Fn']
    phase, constant = params['phase'], params['constant']
    sx = np.array([[0, 1],[ 1, 0]])
    sy = np.array([[0, -1j],[1j, 0]])
    sz = np.array([[1, 0],[0, -1]])

    H_local = np.zeros((2*Fn, 2*Fn), dtype='complex')
    H_pbc = np.zeros((2*Fn, 2*Fn), dtype='complex')
    
    B = t/2 * sz + delta/2 * 1j * sy   
    for i in range(Fn-1):
        if constant == True:
            A_k = mu * sz
        else:
            if AA == False:
                A_k = mu * sz * (1-np.cos(2*np.pi*Fn1/Fn*i+phase))
            else:
                A_k = mu * sz * (np.cos(2*np.pi*Fn1/Fn*i+phase))
        for j in [0,1]:
            for l in [0,1]:
                H_local[2*i+j][2*i+l] = A_k[j][l]
                H_local[2*i+l][2*(i+1)+j] = B[j][l] 
                H_local[2*(i+1)+j][2*i+l] = np.conjugate(B.T[l][j])

    i = Fn-1
    if constant == True:
        A_k = mu * sz
    else:
        if AA == False:
            A_k = mu * sz * (1-np.cos(2*np.pi*Fn1/Fn*i+phase))
        else:
            A_k = mu * sz * (np.cos(2*np.pi*Fn1/Fn*i+phase))
    for j in [0,1]:
        for l in [0,1]:
            H_local[2*i+j][2*i+l] = A_k[j][l]    

    for j in [0,1]:
        for l in [0,1]:    
            H_pbc[2*(Fn-1)+l][j] = B[j][l]  
    
    H_k = H_local + np.exp(1j*k)*H_pbc + np.exp(-1j*k)*H_pbc.T
    
    if rot == True:
        R = rot_sigma_y([0,1,0], np.pi/2, length)
        H_k = R@H_k@np.conjugate(R.T)
  
    return H_k

# OTHER FUNCTIONS

In [ ]:
#export

def Fibonacci(iterations):
    
    """Return Fibonacci sequence number depending on the iterations"""
    
    Fn1 = 0
    Fn = 1
    for n in range(iterations):
        Fn2 = Fn1
        Fn1 = Fn
        Fn = Fn1 + Fn2  
        
    return Fn1, Fn

    
def rot_sigma_y(pvec, psi, Fn):
    
    '''Rotation around sigma y of an angle psi'''
    
    a = np.array([[np.cos(psi/2)-1j*pvec[2]*np.sin(psi/2), (-1j*pvec[0]-pvec[1])*np.sin(psi/2)],
                  [(-1j*pvec[0]+pvec[1])*np.sin(psi/2), np.cos(psi/2)+1j*pvec[2]*np.sin(psi/2)]])
    b = np.identity(Fn)
    
    return np.kron(b,a)


def f(k, alpha):
    
    '''Function f(k,alpha) in terms of the polylog functions'''
    
    f = -1j/2*(polylog(alpha,np.exp(1j*k))-polylog(alpha, np.exp(-1j*k)))
    return float(f.real) + 1j*float(f.imag)


def df(k, alpha):
    
    '''Derivative of function f(k,alpha) in terms of the polylog functions'''
    
    f = 1/2*(polylog(alpha-1,np.exp(1j*k))+polylog(alpha-1, np.exp(-1j*k)))
    return float(f.real) + 1j*float(f.imag)


def HLP(k, alpha, Fn, x, y):
    
    '''Closed form of sum of exponens using HLP function'''
    
    f = lerchphi(np.exp(1j*k),alpha,(Fn+(y-x))/Fn)
    return np.float(re(f))+ 1j*np.float(im(f)) 

In [ ]:
#export

def sparse_diag(matrix, k, sigma=0, **kwargs):
    
    '''Call sla.eigsh with mumps support.'''
    
    class LuInv(sla.LinearOperator):
        def __init__(self, A):
            inst = mumps.MUMPSContext()
            inst.analyze(A, ordering='pord')
            inst.factor(A)
            self.solve = inst.solve
            sla.LinearOperator.__init__(self, A.dtype, A.shape)

        def _matvec(self, x):
            return self.solve(x.astype(self.dtype))
    opinv = LuInv(matrix - sigma * identity(matrix.shape[0]))
    return sla.eigsh(matrix, k, sigma=sigma, OPinv=opinv, **kwargs)